# Seeking out issues

In [1]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_rows', 5000)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
def read(path):
    df = pd.read_csv(path, 
                     header=0, 
                     parse_dates=[
                         'pubTimeStamp', 
#                          'startTime', 
#                          'endTime'
                     ])
    return df

may = read('./data_scooter/may/may_trip.csv')
june = read('./data_scooter/june/june_trip.csv')
july = read('./data_scooter/july/july_trip.csv')

---

## Percentage of scooters reaching 3 or more rides a day

In [8]:
may1 = may.groupby([may.pubTimeStamp.dt.day, 'sumdID']).size()
june1 = may.groupby([may.pubTimeStamp.dt.day, 'sumdID']).size()
july1 = june.groupby([june.pubTimeStamp.dt.day, 'sumdID']).size()

In [9]:
may_gt_3 = (may1 >= 3).sum() 
may_lt_3 = (may1 < 3).sum()

june_gt_3 = (june1 >= 3).sum()
june_lt_3 = (june1 < 3).sum()

july_gt_3 = (july1 >= 3).sum()
july_ls_3 = (july1 < 3).sum()

may_met_3 = (may_gt_3 / (may_gt_3 + may_lt_3) * 100)
june_met_3 = (june_gt_3 / (june_gt_3 + june_lt_3) * 100)
july_met_3 = (july_gt_3 / (july_gt_3 + july_ls_3) * 100)

f"May: {round(may_met_3, 2)}%"
f"June: {round(june_met_3, 2)}%"
f"July: {round(july_met_3, 2)}%"

f"Average: {round(np.mean([may_met_3, june_met_3, july_met_3]))}%"

'May: 43.6%'

'June: 43.6%'

'July: 40.81%'

'Average: 43.0%'

## How many unique sumdID's? 

`Slide says 5860`  
`Below is 9005`


In [25]:
all_months = pd.concat([may, june, july])
all_months.shape
may.shape[0]+june.shape[0]+july.shape[0] # double checking everything concatted

(565522, 16)

565522

In [26]:
print(f"May unique sumdID's: {may.reset_index().sumdID.nunique()}")
print(f"June unique sumdID's: {june.reset_index().sumdID.nunique()}")
print(f"July unique sumdID's: {july.reset_index().sumdID.nunique()}\n")

print(f"Unique sumdID's across all months: {all_months.reset_index().sumdID.nunique()}")

May unique sumdID's: 6966
June unique sumdID's: 6742
July unique sumdID's: 5585

Unique sumdID's across all months: 9005
